In [14]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [15]:
#load data
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [16]:
data

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-10,0,6,1,2,0.0,21.787650,마포구,ST-2167,20221001,2
1,ST-10,1,6,1,1,0.0,20.724511,마포구,ST-2167,20221001,2
2,ST-10,2,6,1,2,0.0,20.222465,마포구,ST-2167,20221001,1
3,ST-10,3,6,1,0,0.0,19.789024,마포구,ST-2167,20221001,0
4,ST-10,4,6,1,2,0.0,20.661831,마포구,ST-2167,20221001,0
...,...,...,...,...,...,...,...,...,...,...,...
9647433,ST-1445,20,2,0,1,0.0,8.507538,용산구,ST-1328,20230228,0
9647434,ST-1445,21,2,0,0,0.0,9.551939,용산구,ST-1328,20230228,3
9647435,ST-1445,22,2,0,0,0.0,8.219911,용산구,ST-1328,20230228,0
9647436,ST-1445,23,2,0,0,0.0,9.181400,용산구,ST-1328,20230228,1


In [17]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [18]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

In [19]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)

### Modeling

In [20]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [21]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [22]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100)
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.461579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1139
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.97516	valid_0's rmse: 1.98775


### Model Evaluation

In [23]:
# 모델 평가
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)

In [24]:
# 훈련 데이터에서의 평가
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)


print(f'Training MSE: {mse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training RMSE: {rmse_train}')


# 검증 데이터에서의 평가
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test RMSE: {rmse_test}')

Training MSE: 3.9012397610454705
Training MAE: 1.145610377419231
Training RMSE: 1.9751556295759254
Test MSE: 3.9511522668585592
Test MAE: 1.1487413989551938
Test RMSE: 1.9877505544857883


### R-Squared

In [25]:
# R-squared 계산
# 훈련 데이터에서의 R-squared 계산
r2_train = r2_score(y_train, y_pred_train)

# 테스트 데이터에서의 R-squared 계산
r2_test = r2_score(y_test, y_pred_test)

print("R-squared for training data:", r2_train)
print("R-squared for test data:", r2_test)

R-squared for training data: 0.5336489160585025
R-squared for test data: 0.5273879188567496
